# Breaking Down Elimination: When It Fails

## <code style="color : Red">1. Permanent Failure (No Solution)</code>
**Characteristics**:
- A row reduces to a contradiction $(0 = b)$ where $b \neq 0$
- System is inconsistent (equations contradict)
- Geometric interpretation: Lines/planes do not intersect

**Example**:
$$
\begin{cases}
x - 2y = 1 & \text{(Equation 1)} \\
3x - 6y = 11 & \text{(Equation 2)}
\end{cases}
$$

**Elimination Steps**:
$$
\begin{aligned}
&\text{Eq2} \leftarrow \text{Eq2} - 3 \times \text{Eq1} \\
&\begin{aligned}
x - 2y &= 1 \\
0 &= 8 \quad \text{(Contradiction)}
\end{aligned}
\end{aligned}
$$

### Code Implementation

In [16]:
import numpy as np

def detect_permanent_failure(A, b):
    """
    Detects if a 2x2 linear system has no solution
    by checking for a contradictory row (i.e., 0x + 0y = nonzero).
    """
    # Copy inputs to avoid altering the originals
    U = A.copy().astype(float)
    c = b.copy().astype(float)

    print("Original system [A | b]:")
    print(np.column_stack((U, c)))
    print()

    # Eliminate the first variable from the second row
    multiplier = U[1, 0] / U[0, 0]
    print(f"Elimination multiplier: {multiplier:.2f}")

    U[1] = U[1] - multiplier * U[0]
    c[1] = c[1] - multiplier * c[0]

    print("\nAfter elimination [U | c]:")
    print(np.column_stack((U, c)))
    print()

    # Check for a row like 0x + 0y = nonzero → contradiction
    if np.allclose(U[1], 0) and not np.isclose(c[1], 0):
        print(f"Contradiction found: 0 = {c[1]:.2f}")
        print("→ The system is inconsistent and has no solution.")
        return True
    else:
        print("No contradiction detected. The system may have a unique or infinite solution.")
        return False

# Example usage
A = np.array([[1, -2], 
              [3, -6]], dtype=float)  # Second row is a multiple of the first
b = np.array([1, 11], dtype=float)    # But with a different RHS → contradiction

detect_permanent_failure(A, b)

Original system [A | b]:
[[ 1. -2.  1.]
 [ 3. -6. 11.]]

Elimination multiplier: 3.00

After elimination [U | c]:
[[ 1. -2.  1.]
 [ 0.  0.  8.]]

Contradiction found: 0 = 8.00
→ The system is inconsistent and has no solution.


True

---

## <code style="color : Red">2. Degenerate Case (Infinitely Many Solutions)
**Characteristics**:
- A row reduces to $0 = 0$ (redundancy)
- Equations are linearly dependent
- System has free variables → infinite solutions

**Example**:
$$
\begin{cases}
x - 2y = 1 & \text{(Equation 1)} \\
3x - 6y = 3 & \text{(Equation 2)}
\end{cases}
$$

**Elimination Steps**:
$$
\begin{aligned}
&\text{Eq2} \leftarrow \text{Eq2} - 3 \times \text{Eq1} \\
&\begin{aligned}
x - 2y &= 1 \\
0 &= 0 \quad \text{(Redundancy)}
\end{aligned}
\end{aligned}
$$

**Pivot Count**: Only 1 pivot remains

### Code Implementation

In [15]:
import numpy as np

def detect_infinite_solutions(A, b):
    """
    Detects if a 2x2 linear system has infinitely many solutions
    by checking for a redundant row (i.e., 0 = 0 after elimination).
    """
    # Copy input to avoid changing the original data
    U = A.copy().astype(float)
    c = b.copy().astype(float)

    print("Original system [A | b]:")
    print(np.column_stack((U, c)))
    print()

    # Eliminate the first variable from the second row
    multiplier = U[1, 0] / U[0, 0]
    print(f"Elimination multiplier to zero out A[1,0]: {multiplier:.2f}")
    
    U[1] = U[1] - multiplier * U[0]
    c[1] = c[1] - multiplier * c[0]

    print("\nAfter elimination [U | c]:")
    print(np.column_stack((U, c)))
    print()

    # Check if the second row is 0 = 0 (i.e., redundant)
    if np.allclose(U[1], 0) and np.isclose(c[1], 0):
        print("This is a degenerate case: the second equation is redundant.")
        print("→ The system has infinitely many solutions.")
        return True
    else:
        print("No redundancy detected. The system does not have infinite solutions.")
        return False

# Example usage
A = np.array([[1, -2],
              [3, -6]], dtype=float)  # Second row is 3× the first row
b = np.array([1, 3], dtype=float)     # RHS also scaled by 3

detect_infinite_solutions(A, b)

Original system [A | b]:
[[ 1. -2.  1.]
 [ 3. -6.  3.]]

Elimination multiplier to zero out A[1,0]: 3.00

After elimination [U | c]:
[[ 1. -2.  1.]
 [ 0.  0.  0.]]

This is a degenerate case: the second equation is redundant.
→ The system has infinitely many solutions.


True

---

## <code style="color : Orange">3. Temporary Failure (Zero Pivot)
**Characteristics**:
- Zero appears in pivot position
- Fixable via row exchange

**Example**:
$$
\begin{cases}
0x + 2y = 4 & \text{(Equation 1)} \\
3x - 2y = 5 & \text{(Equation 2)}
\end{cases}
$$

**Solution**: Row permutation
$$
\begin{cases}
3x - 2y = 5 \\
2y = 4
\end{cases}
$$

**Result**: Proper upper triangular system after swap

### Code Implementation

In [14]:
import numpy as np

def handle_zero_pivot(A, b):
    """
    Checks if the pivot in the first row is zero.
    If so, swaps the first two rows of A and b to continue Gaussian elimination.
    Then performs one step of elimination.
    """
    # Copy inputs to avoid modifying original data
    U = A.copy().astype(float)
    c = b.copy().astype(float)

    print("Initial system [A | b]:")
    print(np.column_stack((U, c)))
    print()

    if np.isclose(U[0, 0], 0):
        print("Zero pivot detected at A[0, 0]. Swapping row 0 and row 1...\n")
        
        # Swap first and second rows
        U[[0, 1]] = U[[1, 0]]
        c[[0, 1]] = c[[1, 0]]

        print("After row swap [A | b]:")
        print(np.column_stack((U, c)))
        print()

    # Perform elimination to zero out U[1, 0]
    multiplier = U[1, 0] / U[0, 0]
    U[1] = U[1] - multiplier * U[0]
    c[1] = c[1] - multiplier * c[0]

    print(f"Elimination multiplier: {multiplier:.2f}")
    print("\nSystem after elimination step [U | c]:")
    print(np.column_stack((U, c)))

# Example usage
A = np.array([[0, 2],
              [3, -2]])
b = np.array([4, 5])

handle_zero_pivot(A, b)

Initial system [A | b]:
[[ 0.  2.  4.]
 [ 3. -2.  5.]]

Zero pivot detected at A[0, 0]. Swapping row 0 and row 1...

After row swap [A | b]:
[[ 3. -2.  5.]
 [ 0.  2.  4.]]

Elimination multiplier: 0.00

System after elimination step [U | c]:
[[ 3. -2.  5.]
 [ 0.  2.  4.]]


---

## <code style="color : Green">Key Observations
| Failure Type         | Algebraic Sign | Geometric Meaning          | Resolution         |
|----------------------|----------------|----------------------------|--------------------|
| Permanent            | \( 0 = b \)    | No intersection            | No solution        |
| Degenerate           | \( 0 = 0 \)    | Infinite intersections     | Free variables     |
| Temporary (Zero Pivot)| Zero pivot     | Needs reorientation        | Row exchange       |